# セクター別リターン

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hiohiohio/jquants-api-client-python/blob/feature/add-sample-notebooks/examples/20220825-002-sector.ipynb)

このノートブックではセクター別のリターンを算出して表示します。

1. 過去6ヶ月および過去3ヶ月のセクター別リターン
2. セクター別の月次リターン


----

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のファイルにリフレッシュトークンが書き込まれていることを想定しています。
    - `MyDrive/drive_ws/secret/jquantsapi-key.txt`
- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [ ]:
# 必要なモジュールをインストールします。
! python -m pip install jquants-api-client japanize-matplotlib

In [ ]:
# Google drive をマウントします。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from datetime import datetime

import japanize_matplotlib
import jquantsapi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

In [ ]:
# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [ ]:
# プロット用の設定をします
sns.set(rc={'figure.figsize': (15, 10)})
sns.set(font_scale=2)
sns.set_style('whitegrid')

japanize_matplotlib.japanize()

In [ ]:
# 一度取得したデータは Google Drive 上に保存して再利用します。
# 保存先ディレクトリを指定します。
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)
STORAGE_DIR_PATH

In [ ]:
# リフレッシュトークンが記載されているファイルを指定します
REFRESH_TOKEN_FILE_PATH = "/content/drive/MyDrive/drive_ws/secret/jquantsapi-key.txt"

In [ ]:
# リフレッシュトークンを読み込むための関数を定義します
def get_refresh_token(refresh_token_file_path: str = REFRESH_TOKEN_FILE_PATH):
    with open(refresh_token_file_path, "r") as f:
        refresh_token = f.read()
    return refresh_token.rstrip().lstrip()

In [ ]:
# リフレッシュトークンをファイルから読み込みます
refresh_token = get_refresh_token()

In [ ]:
# J-Quants APIのクライアントクラスを初期化します
jqapi = jquantsapi.Client(refresh_token=refresh_token)

In [ ]:
# 銘柄情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
if now.hour < 22:
    # データ更新時間前の場合は日付を1日ずらします。
    now -= pd.Timedelta(1, unit="D")
list_file = f"{STORAGE_DIR_PATH}/list_{now.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(list_file):
    # 保存されているデータが存在しない場合はAPIからデータを取得します
    df_list = jqapi.get_list()
    # 取得したデータをファイルに保存します
    df_list.to_csv(list_file, compression="gzip", index=False)
    print(f"save file: {list_file}")

# ファイルからデータを読み込みます
print(f"file exists: {list_file}, loading")
df_list = pd.read_csv(list_file, dtype="str")

In [ ]:
# データを表示して取得できていることを確認します
df_list.tail(2)

In [ ]:
# 市場区分別の銘柄数を把握します。
df_list.groupby(["MarketCode", "MarketName"]).size().reset_index().sort_values([0], ascending=False).rename(columns={0: "銘柄数"}).set_index("MarketName").plot(kind="bar", figsize=(10, 10), grid=True)

In [ ]:
# セクター別の銘柄数を把握します。
df_list.groupby(["SectorCode", "SectorName"]).size().reset_index().set_index("SectorName").sort_values(0, ascending=False).rename(columns={0: "銘柄数"}).plot(kind="bar", figsize=(20, 8), grid=True)

In [ ]:
%%time
# 株価情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
# 過去6ヶ月のデータを取得
start_dt = now - pd.Timedelta(190, unit="D")  # 計算用に10日分多めに指定しています
end_dt = now
if end_dt.hour < 19:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")
price_file = f"{STORAGE_DIR_PATH}/price_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(price_file):
    df_p = jqapi.get_price_range(start_dt=start_dt, end_dt=end_dt)
    df_p.to_csv(price_file, compression="gzip", index=False)
    print(f"save file: {price_file}")

# データを読み込みます
print(f"file exists: {price_file}, loading")
df_p = pd.read_csv(price_file, dtype="str")
df_p.reset_index(drop=True, inplace=True)
# 各列のデータ型を調整します
df_p.loc[:, "Date"] = pd.to_datetime(df_p["Date"], format="%Y-%m-%d")
df_p.loc[:, "Open"] = df_p["Open"].astype(np.float64)
df_p.loc[:, "High"] = df_p["High"].astype(np.float64)
df_p.loc[:, "Low"] = df_p["Low"].astype(np.float64)
df_p.loc[:, "Close"] = df_p["Close"].astype(np.float64)
df_p.loc[:, "Volume"] = df_p["Volume"].astype(np.float64)
df_p.loc[:, "TurnoverValue"] = df_p["TurnoverValue"].astype(np.float64)
df_p.loc[:, "AdjustmentFactor"] = df_p["AdjustmentFactor"].astype(np.float64)
df_p.loc[:, "AdjustmentOpen"] = df_p["AdjustmentOpen"].astype(np.float64)
df_p.loc[:, "AdjustmentHigh"] = df_p["AdjustmentHigh"].astype(np.float64)
df_p.loc[:, "AdjustmentLow"] = df_p["AdjustmentLow"].astype(np.float64)
df_p.loc[:, "AdjustmentClose"] = df_p["AdjustmentClose"].astype(np.float64)
df_p.loc[:, "AdjustmentVolume"] = df_p["AdjustmentVolume"].astype(np.float64)

In [ ]:
# 取得したデータを確認します
df_p.tail(2)

In [ ]:
# 直近3ヶ月および6ヶ月のセクター別リターンをプロットします

# データから最も古い日付を取得します
base_date = df_p["Date"].min()
# データから最新日付を取得します
evaluation_date = df_p["Date"].max()
# 6ヶ月前の日付を取得します
base_6months_date = df_p.loc[df_p["Date"] <= (evaluation_date - pd.Timedelta(180, unit="D")).to_datetime64(), "Date"].max()
# ３ヶ月前の日付を取得します
base_3months_date = df_p.loc[df_p["Date"] <= (evaluation_date - pd.Timedelta(90, unit="D")).to_datetime64(), "Date"].max()

In [ ]:
# 取引がなかった日付の終値には0が含まれていますがそのままでは計算しずらいため
# 終値が0の場合は前営業日の終値を使用します
df_p.sort_values(["Code", "Date"], inplace=True)
df_p["AdjustmentClose"].replace({0.0: np.nan}, inplace=True)
df_p.loc[:, "AdjustmentClose"] = df_p.groupby("Code")["AdjustmentClose"].ffill()
# 終値がnanの場合は翌営業日の終値を使用します (データの先頭をフィルするため)
df_p.loc[:, "AdjustmentClose"] = df_p.groupby("Code")["AdjustmentClose"].bfill()


# 基準日の株価を取得します
filter_base_date = df_p["Date"] == base_date
df_base = df_p.loc[filter_base_date]

# 6ヶ月前の株価を取得します
filter_6months_date = df_p["Date"] == base_6months_date
df_6months = df_p.loc[filter_6months_date]

# 3ヶ月前の株価を取得します
filter_3months_date = df_p["Date"] == base_3months_date
df_3months = df_p.loc[filter_3months_date]

# 評価日の株価を取得します
filter_eval_date = df_p["Date"] == evaluation_date
df_eval = df_p.loc[filter_eval_date]

# データを結合する際に必要なカラムを指定します
cols = ["Code", "Date", "AdjustmentClose"]

# 評価日の株価をベースに作業します
df_work = df_eval[cols]
# 基準日の株価と結合します
df_work = pd.merge(df_work, df_base[cols], on=["Code"], how="left", suffixes=("", "_base"))
# 6ヶ月前の株価と結合します
df_work = pd.merge(df_work, df_6months[cols], on=["Code"], how="left", suffixes=("", "_6months"))
# 3ヶ月前の株価と結合します
df_work = pd.merge(df_work, df_3months[cols], on=["Code"], how="left", suffixes=("", "_3months"))

# 6ヶ月間のリターンを算出します
df_work["return_6months"] = ((df_work["AdjustmentClose"] / df_work["AdjustmentClose_6months"]) - 1) * 100
# 3ヶ月間のリターンを算出します
df_work["return_3months"] = ((df_work["AdjustmentClose"] / df_work["AdjustmentClose_3months"]) - 1) * 100

# 月末営業日を取得します
eoms = df_p.groupby(pd.Grouper(key="Date", freq="M")).apply(lambda df: df["Date"].max()).tolist()
# 月末の株価と結合してリターンを算出します
prev_ym = eoms[0].strftime("%Y%m")
for i, eom in enumerate(eoms):
  ym = eom.strftime("%Y%m")
  filter_eom = df_p["Date"] == eom
  df = df_p.loc[filter_eom]
  df_work = pd.merge(df_work, df[cols], on=["Code"], how="left", suffixes=("", f"_{ym}"))
  if i != 0:
      df_work[f"return_monthly_{ym}"] = ((df_work[f"AdjustmentClose_{ym}"] / df_work[f"AdjustmentClose_{prev_ym}"]) - 1) * 100
  prev_ym = ym

# セクター情報と結合します
df_work = pd.merge(df_work, df_list[["Code", "SectorCode", "SectorName", "MarketCode"]], on=["Code"])

# プライム銘柄に絞り込みます
df_work = df_work.loc[df_work["MarketCode"] == "A"]

# 6ヶ月、3ヶ月、および基準日を出力します
display(base_6months_date, base_3months_date, evaluation_date)

# セクター別のリターンをプロットします
df_work.groupby("SectorName")["return_6months", "return_3months"].mean().sort_values("return_6months", ascending=False).plot(kind="bar", figsize=(20, 8), grid=True)

In [ ]:
# 月毎のリターンの項目名を取得します
monthly_cols = sorted([c for c in df_work.columns if c.startswith("return_monthly_")])
# セクター別のリターンを算出します
df_monthly = df_work.groupby("SectorName")[monthly_cols].mean()
# セクター別のリターンを月毎に表示します (最新月は月初から当日までのリターンになっています)
df_monthly.sort_values(monthly_cols[-1], ascending=False).style.background_gradient(cmap="RdYlGn", axis=0)